In [1]:
from os import path 

from keras.layers import Input
from keras.models import Model

import numpy as np

import blusky.datasets as datasets
from blusky.transforms.apply_father_wavelet_2d import ApplyFatherWavlet2D
from blusky.transforms.cascade_2d import Cascade2D
from blusky.transforms.cascade_tree import CascadeTree
from blusky.transforms.default_decimation import DefaultDecimation, NoDecimation
from blusky.utils.pad_2d import Pad2D, pad_to_log2
from blusky.wavelets.wavelet_factories_2d import vanilla_gabor_2d, vanilla_morlet_2d

from scipy.signal import convolve2d

Using TensorFlow backend.


In [2]:
test_file_1 = path.join(
    path.dirname(datasets.__file__), "twod_image_1.npy")

original_image = np.load(test_file_1)
original_shape = original_image.shape

# get a single tile from the image to test
# note this image is currently unpadded.  
# how many boundary elements are needed to pad?
extracted_image = original_image[0:32,0:32]
extracted_shape = extracted_image.shape

img = np.expand_dims(extracted_image, axis=-1)

In [3]:
# Don't make this too huge for brevity.
J = 3
# 0 = no overlap etc.
overlap_log_2 = 0
# apply to all available orders
order = J
# Should be one or more to avoid aliasing, if you want overlapping tiles, 
# this can increase too.
oversampling = 0

angles=(0.0, 45., 90.)


# details of the input data
img_size = img.shape
sample_rate = 0.004*3

# vanilla filter bank
wavelets = [vanilla_morlet_2d(sample_rate, j=i) for i in range(0,J)]
father_wavelet = vanilla_gabor_2d(sample_rate, j=J)
print (father_wavelet.kernel(0.).shape)
# method of decimation
deci = NoDecimation() # DefaultDecimation(oversampling=oversampling)

# input
inp = Input(shape=img.shape)

# valid padding
cascade2d = Cascade2D("none", 
                    0, 
                    decimation=deci,
                    angles=angles)

# Pad the input
pad_2d = Pad2D(wavelets, decimation=deci)
padded = pad_2d.pad(inp)

# Apply cascade with successive decimation
cascade_tree = CascadeTree(padded, order=order)
cascade_tree.generate(wavelets, cascade2d._convolve)
convs = cascade_tree.get_convolutions()

# Create layers to remove padding
cascade_tree = CascadeTree(padded, order=order)
cascade_tree.generate(wavelets, pad_2d._unpad_same)
unpad = cascade_tree.get_convolutions()

# Remove the padding
unpadded_convs = [i[1](i[0]) for i in zip(convs, unpad)]

# Complete the scattering transform with the father wavelet
apply_conv = ApplyFatherWavlet2D(J=J, overlap_log_2=overlap_log_2, 
                              img_size=img.shape, sample_rate=sample_rate,
                              wavelet=father_wavelet)

sca_transf = apply_conv.convolve(unpadded_convs)

W1009 13:52:10.955075 4640142784 deprecation_wrapper.py:119] From /Users/bhall/.edm/envs/blusky/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1009 13:52:10.969797 4640142784 deprecation_wrapper.py:119] From /Users/bhall/.edm/envs/blusky/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1009 13:52:10.995784 4640142784 deprecation.py:323] From /Users/bhall/.edm/envs/blusky/lib/python3.6/site-packages/tensorflow/python/ops/variables.py:2618: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


(31, 31)


In [4]:
model = Model(inputs=inp, outputs=sca_transf)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
reflection_padding2d_1 (Reflect (None, 64, 64, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
depthwise_conv2d_1 (DepthwiseCo (None, 64, 64, 3)    150         reflection_padding2d_1[0][0]     
__________________________________________________________________________________________________
depthwise_conv2d_2 (DepthwiseCo (None, 64, 64, 3)    150         reflection_padding2d_1[0][0]     
__________________________________________________________________________________________________
lambda_1 (

In [5]:
result = model.predict(np.expand_dims(img, axis=0))

W1009 13:52:18.128229 4640142784 deprecation_wrapper.py:119] From /Users/bhall/.edm/envs/blusky/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1009 13:52:18.132870 4640142784 deprecation_wrapper.py:119] From /Users/bhall/.edm/envs/blusky/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1009 13:52:18.134224 4640142784 deprecation_wrapper.py:119] From /Users/bhall/.edm/envs/blusky/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



The first result is $\left|x * \psi_{\lambda_1}\right|*\phi$.  Compute this manually.

In [26]:
# extract the kernels of each of the wavelets for manual convolution
# we'll test using three different angles that we used to create the
# transform above.
wav1 = wavelets[0]
wav2 = wavelets[1]
wav3 = wavelets[2]

# extract the kernels of each of the wavelets for manual convolution
# we'll test using three different angles that we used to create the
# transform above.
wav1_k = wav1.kernel(0.0)
wav2_k = wav2.kernel(45.0)
wav3_k = wav3.kernel(90.0)

phi = father_wavelet.kernel(0.)


img_pad = np.pad(img, ((16,16), (16,16), (0,0)), mode='reflect')
# get numpy array of the test input image
x = img_pad[:,:,0]

# manual convolution, |x * psi_1|
conv = np.abs(convolve2d(x, wav1_k, mode="same"))


conv2 = np.abs(convolve2d(conv, wav2_k, mode="same"))

conv3 = np.abs(convolve2d(conv2, wav3_k, mode="same"))


In [27]:
manual_result1 = convolve2d(conv[16:-16, 16:-16], phi.real, mode="valid")[0,0]
manual_result2 = convolve2d(conv2[16:-16, 16:-16], phi.real, mode="valid")[0,0]
manual_result3 = convolve2d(conv3[16:-16, 16:-16], phi.real, mode="valid")[0,0]

print('manual result 1: {}'.format(manual_result1))
print('manual result 2: {}'.format(manual_result2))
print('manual result 3: {}'.format(manual_result3))

manual result 1: 84.78744319603979
manual result 2: 12.140577175061448
manual result 3: 1.3845558275244014


find this in the output.  it should be something like xpsi3unpaddedphi

In [28]:
cnn_result1 = result[0][0,0,0,0]
cnn_result1

86.09375

In [30]:
cnn_result2 = result[3][0,0,0,1]
cnn_result2

12.226252

In [29]:
cnn_result3 = result[6][0,0,0,5]
cnn_result3

1.3982526

In [17]:
result

[array([[[[ 86.09375, 141.99812, 673.1921 ]]]], dtype=float32),
 array([[[[ 43.508118,  91.91322 , 488.20453 ]]]], dtype=float32),
 array([[[[ 24.942146,  63.18323 , 445.87457 ]]]], dtype=float32),
 array([[[[  9.459414,  12.226252,  10.914241,  11.449362,  15.020585,
            18.954191,  16.832403,  36.586838, 100.179825]]]],
       dtype=float32),
 array([[[[  5.7497225,  12.36543  ,  12.172748 ,   8.588438 ,
            12.484282 ,  21.519789 ,  15.416336 ,  49.274372 ,
           115.93939  ]]]], dtype=float32),
 array([[[[  4.8478727,   4.557818 ,   4.97699  ,   9.299008 ,
             6.3051643,  10.920145 ,   8.978993 ,  20.972616 ,
           105.831604 ]]]], dtype=float32),
 array([[[[ 1.615682  ,  1.2423929 ,  0.96118116,  1.5900751 ,
            2.0602398 ,  1.3982526 ,  0.5933875 ,  0.97910297,
            1.270066  ,  1.601808  ,  0.78247935,  1.2253474 ,
            1.1028441 ,  1.5106843 ,  1.6233064 ,  0.6842292 ,
            0.88154525,  2.2993293 ,  2.2082953 ,  1.